In [1]:
import requests
from psycopg2.extensions import AsIs
import pandas as pd
from time import sleep
from datetime import datetime
from config import vk_access_token, db_db_name, db_host, db_password, db_user
from sqldb import DataBase_Con


In [2]:
connect_db = DataBase_Con(db_host,db_user,  db_password,  db_db_name)
connect_db.connection_on()

The connnection in ON


In [3]:

def getVkJson(list_of_indexes) -> pd.DataFrame:
    df = pd.DataFrame()
    for x in list_of_indexes:
        url = 'https://api.vk.com/method/'
        method_wall = 'wall.get'
        v = '5.131'
        params_wall = {
        'owner_id':x[0]*-1,
        'access_token':vk_access_token,
        'count': 30,
        'v':v
    }
        req_wall = requests.get(url + method_wall, params = params_wall, allow_redirects=False)
        req_wall.close
        sleep(0.5)
        df = pd.concat([df, pd.DataFrame(req_wall.json()['response']['items'])])

    return df.reset_index()


def dateStamper(arr):
    return pd.to_datetime(datetime.utcfromtimestamp(arr).strftime('%Y-%m-%d %H:%M:%S'))

def currentDate():
    return pd.to_datetime(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


def changeSignOwnerID(arr):
    return arr *-1


def getPhotoMetaData(arr) -> dict:
    if 'photo' in arr[0].values():
        return arr[0]['photo']


def getAttachments(arr) -> dict:
    if 'photo' in arr[0].values():
        return arr[0]['photo']['sizes']

def getPhotoID(arr) -> dict:
    if 'post_id' in arr.keys() :
        return arr['post_id']


def getSizes(arr) -> pd.DataFrame:
    temp_list = []
    for ind in range(len(arr)):
        for x in arr['meta_photo'][ind]['sizes']:
            if 'x' in x.values():
                temp_list.append( x['url'])

    return pd.DataFrame(temp_list, columns=['url'])


def convert_asis(arr):
    return AsIs(arr)


def dateSTAMP(arr):
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')





In [4]:
main_list = connect_db.getIndexesFromMainDB()
df_from_vk = getVkJson(main_list)

[INFO]Groups has been gotten. Func getIndexesFromMainDB has finished


In [5]:
def currentDate():
    return pd.to_datetime(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [6]:
def mainColumns(func) -> pd.DataFrame:
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        return result[['id','owner_id','text','date','attachments']]
    return inner

In [7]:
@mainColumns
def requiredData(df) -> pd.DataFrame:
    return df.loc[(df['carousel_offset'] != 0.0)&(df['marked_as_ads'] != 1)&(df['attachments'] != None)]


In [8]:
frame_main_df = requiredData(df_from_vk)[:]

In [9]:
def getdicts(func):
    def get0(df):
        result = func(df)
        try:
            return result.get('sizes')
        except:
            return None        
    return get0

In [10]:
def gettingUrl(func):
    def inner(*args, **kwargs):
        result = func(*args, **kwargs)
        try:
            for x in result:
                if  x.get('type') == 'x':
                    return x['url']
        except:
            return None
    return inner
    

In [11]:
@gettingUrl
@getdicts
def unpaker(df):
    try:
        return df[0].get('photo')
    except:
        return None

    

In [12]:
# def changetype(func) -> pd.DataFrame:
#     def inner(df):
#     new_df = df.rename(columns = {'id':'content_id', 'date':'public_date'})
#     return new_df.astype({'owner_id':int,'public_date':str, 'save_date':str})
    

In [13]:
def changetype(func) -> pd.DataFrame:
    def inner(df):
        result = func(df)
        new_df = result.rename(columns = {'id':'content_id', 'date':'public_date'})
        return new_df.astype({'owner_id':int,'public_date':str, 'save_date':str})
    return inner
    

In [14]:
@changetype
def properformat(df) -> pd.DataFrame:
    new_df = df.dropna().reset_index().drop(columns = ['index','attachments'])
    new_df['owner_id'] = new_df['owner_id'] *-1
    return new_df

In [15]:
def dateStamper(arr):
    return pd.to_datetime(datetime.utcfromtimestamp(arr).strftime('%Y-%m-%d %H:%M:%S'))

In [16]:
frame_main_df['url'] = frame_main_df['attachments'].apply(unpaker)


In [17]:
frame_main_df['date'] = frame_main_df['date'].apply(dateStamper)
frame_main_df['save_date'] = currentDate()

In [18]:
data_to_send = properformat(frame_main_df)[:]

In [19]:
data_to_send.head(1)

,content_id,owner_id,text,public_date,url,save_date
0,12180316,460389,,2022-05-11 07:37:33,https://sun9-57.userapi.com/s/v1/if2/Xlb_ab-16...,2022-05-13 15:25:10


In [20]:
temp_df = connect_db.GetIDPostsSQL()
try:
    new_df = takeNewestPosts(temp_df, data_to_send)

    for_content_db = new_df[['content_id', 'owner_id', 'public_date', 'save_date']]
    for_post_db = new_df[[ 'url', 'text','content_id']]
    for_content_db = for_content_db.astype({'public_date':str, 'save_date':str})
    connect_db.sendDataContentSQL(for_content_db)
    connect_db.sendDataPostsSQL(for_post_db)
    print("[INFO] Databases have been updated")
except Exception as _ex:
    print("[INFO] You don't have a new data", _ex)

[INFO]content_db has been gotten. Func GetIDPostsSQL has finished
[INFO] Content_db has been updated. Func sendDataContentSQL has finished
[INFO] Post_db has been updated. Func sendDataPostsSQL has finished
[INFO] Databases have been updated
